In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import string 
import re
import string
from datetime import datetime, date, time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

Parsing HTML for data so
We get the raw HTML from Space Weather live Website 
and use an HTML parsing library called BeautifulSoup 
to extract the data of interest.

 ------------------------------------------ Part 1 content ------------------------------------------

In [2]:

url = 'https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares'
content = requests.get(url).content
root = BeautifulSoup(content, 'html.parser')

In [3]:
'''
Finding the Data with tag "table" that its class like shown below
then we find all childern of it with tag 'td'
'''
Table = root.find('table', class_ = 'table table-striped table-responsive-md').findAll('td')
#Get text from the Data we get to get rid of <html> tags and other unnecessary
MyData = [pt.get_text() for pt in Table]
for i in MyData:
    if i == ' MovieView archive' or i == 'View archive':
        MyData.remove(i)
#-----------------------------------------------------------------

RankList = []
X_ClassList = []
DateList = []
RegionList = []
StartTimeList = []
MaxTimeList = []
EndTimeList = []

#We append the data in lists to put it in the DataFrame 
for i in range(0, len(MyData), 7):
    if i + 6 < len(MyData):
        RankList.append(MyData[i])
        X_ClassList.append(MyData[i + 1])
        DateList.append(MyData[i + 2])
        RegionList.append(MyData[i + 3])
        StartTimeList.append(MyData[i + 4])
        MaxTimeList.append(MyData[i + 5])
        EndTimeList.append(MyData[i + 6])
#-----------------------------------------------------------------
#We Parse the Date and time from str to Datetime objects 
NStart_TimeDF = []
for i in StartTimeList:
    start_hrs, start_min = i.split(':')
    t = time(int(start_hrs), int(start_min))
    NStart_TimeDF.append(t)
    
NMax_TimeDF = []
for i in MaxTimeList:
    start_hrs, start_min = i.split(':')
    t = time(int(start_hrs), int(start_min))
    NMax_TimeDF.append(t)

NEnd_TimeDF = []
for i in EndTimeList:
    start_hrs, start_min = i.split(':')
    t = time(int(start_hrs), int(start_min))
    NEnd_TimeDF.append(t)
    
NEnd_DateDF = []
for i in DateList:
    years, months, days = i.split('/')
    d = date(int(years), int(months), int(days))
    NEnd_DateDF.append(d)

In [4]:
#Put the lists in the dataframe
data = {'Rank':RankList,
        'X_class':X_ClassList,
        'Date':NEnd_DateDF,
        'Region':RegionList,
        'Start_Time':NStart_TimeDF,
        'Max_Time':NMax_TimeDF,
        'End_Time':NEnd_TimeDF}

df = pd.DataFrame(data)

#Zipping the column of Date with Start time, Max time and End time
df['Start_Time'] = list(zip(df.iloc[:,2], df.iloc[:,4]))
df['Max_Time'] = list(zip(df.iloc[:,2], df.iloc[:,5]))
df['End_Time'] = list(zip(df.iloc[:,2], df.iloc[:,6]))

#Drop the Date column because its unnecessary now
df = df.drop('Date', axis = 1)

df.head(50)

,Rank,X_class,Region,Start_Time,Max_Time,End_Time
0,1,X28+,0486,"(2003-11-04, 19:29:00)","(2003-11-04, 19:53:00)","(2003-11-04, 20:06:00)"
1,2,X20+,9393,"(2001-04-02, 21:32:00)","(2001-04-02, 21:51:00)","(2001-04-02, 22:03:00)"
2,3,X17.2+,0486,"(2003-10-28, 09:51:00)","(2003-10-28, 11:10:00)","(2003-10-28, 11:24:00)"
3,4,X17+,0808,"(2005-09-07, 17:17:00)","(2005-09-07, 17:40:00)","(2005-09-07, 18:03:00)"
4,5,X14.4,9415,"(2001-04-15, 13:19:00)","(2001-04-15, 13:50:00)","(2001-04-15, 13:55:00)"
5,6,X10,0486,"(2003-10-29, 20:37:00)","(2003-10-29, 20:49:00)","(2003-10-29, 21:01:00)"
6,7,X9.4,8100,"(1997-11-06, 11:49:00)","(1997-11-06, 11:55:00)","(1997-11-06, 12:01:00)"
7,8,X9.3,2673,"(2017-09-06, 11:53:00)","(2017-09-06, 12:02:00)","(2017-09-06, 12:10:00)"
8,9,X9,0930,"(2006-12-05, 10:18:00)","(2006-12-05, 10:35:00)","(2006-12-05, 10:45:00)"
9,10,X8.3,0486,"(2003-11-02, 17:03:00)","(2003-11-02, 17:25:00)","(2003-11-02, 17:39:00)"


 ------------------------------------------ Part 2 Content -----------------------------------------------------

In [6]:
#Parsing HTML for data
'''
We get the raw HTML from Space Weather live Website 
and use an HTML parsing library called BeautifulSoup 
to extract the data of interest.
'''
url2 = 'https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html'
content2 = requests.get(url2)
root2 = BeautifulSoup(content2.text)

#read the tag 'pre' because the tag 'a' has previous siblings
#so its more easier to read the parent tag 'pre'
Table = root2.find('pre')
Table = Table.text
FinalTable = Table.splitlines()
FinalTable = FinalTable[12:]

In [7]:
Start_Date = []
Start_Time = []
End_Date = []
End_Time = []
Start_Freq = []
End_Freq = []
Flare_Location = []
Flare_region = []
Importance = []
CME_Date = []
CME_Time = []
CPA = []
Width = []
Speed = []
Plots = []
is_Halo = []
Width_Column = []

#appending the data in lists to put it in the DataFrame using slices 
#use .strip() function to get rid of unwanted spaces
for i in range(0, len(FinalTable)-1):
    Start_Date.append(FinalTable[i][0:11].strip())
    Start_Time.append(FinalTable[i][11:16].strip())
    End_Date.append(FinalTable[i][16:22].strip())
    End_Time.append(FinalTable[i][22:28].strip())
    Start_Freq.append(FinalTable[i][28:35].strip())
    End_Freq.append(FinalTable[i][35:40].strip())
    Flare_Location.append(FinalTable[i][40:50].strip())
    Flare_region.append(FinalTable[i][50:55].strip())
    Importance.append(FinalTable[i][55:60].strip())
    CME_Date.append(FinalTable[i][60:69].strip())
    CME_Time.append(FinalTable[i][69:75].strip())
    CPA.append(FinalTable[i][75:80].strip())
    Width.append(FinalTable[i][80:85].strip())
    Speed.append(FinalTable[i][85:90].strip())
    Plots.append('PHTX')

#We Parse the Date and time from str to Datetime objects 
CMEE_Date = []
for i in CME_Date:
    if i == '--/--':
        CMEE_Date.append('NaN')
    else:
        CMEE_Date.append(i)
CMEE_Time = []
for i in CME_Time:
    if i == '--:--':
        CMEE_Time.append('NaN')
    else:
        CMEE_Time.append(i)
NCPA = []
for i in CPA:
    if i == 'Halo':
        NCPA.append('NaN')
        is_Halo.append('True')
    elif i == '----':
        NCPA.append('NaN')
        is_Halo.append('False')
    else:
        NCPA.append(i)
        is_Halo.append('False')
NWidth_Column = []
for i in Width:
    if i.startswith('>'):
        NWidth_Column.append(i[1:])
        Width_Column.append('True')
    else:
        NWidth_Column.append(i)
        Width_Column.append('False')
NStart_Date= []

MyDate = []
for i in Start_Date:
    year, months, days = i.split('/')
    MyDate.append(int(year))
    d = date(int(year), int(months), int(days))
    NStart_Date.append(d)
    
NStart_Time = []
for i in Start_Time:
    start_hrs, start_min = i.split(':')
    t = time(int(start_hrs), int(start_min))
    NStart_Time.append(t)
    
NEnd_Date = []
for i in range(0, len(End_Date)):
    months, days = End_Date[i].split('/')
    d = date(MyDate[i], int(months), int(days))
    NEnd_Date.append(d)
    
NEnd_Time = []
for i in End_Time:
    start_hrs, start_min = i.split(':')
    if int(start_hrs) == 24:
        start_hrs = int('00')
    t = time(int(start_hrs), int(start_min))
    NEnd_Time.append(t)
    
NCME_Time= []
for i in CMEE_Time:
    if i == 'NaN':
        NCME_Time.append('NaN')
    else:
        start_hrs, start_min = i.split(':')
        t = time(int(start_hrs), int(start_min))
        NCME_Time.append(t)
NCME_Date = []
for i in range(0, len(CMEE_Date)):
    if CMEE_Date[i] == 'NaN':
        NCME_Date.append('NaN')
    else:    
        months, days = CMEE_Date[i].split('/')
        d = date(MyDate[i], int(months), int(days))
        NCME_Date.append(d)


In [8]:
#Put the lists in the dataframe
data2= {'Start_Date':NStart_Date,
        'Start_Time':NStart_Time,
        'End_Date':NEnd_Date,
        'End_Time':NEnd_Time,
        'Start_Freq':Start_Freq,
        'End_Freq':End_Freq,
        'Flare_Location':Flare_Location,
        'Flare_region':Flare_region,
        'Importance':Importance,
        'CME_Date':NCME_Date,
        'CME_Time':NCME_Time,
        'CPA':NCPA,
        'Width':NWidth_Column,
        'Speed':Speed,
        'Plots':Plots,
        'is_Halo':is_Halo,
        'WidthColumn':Width_Column}

df2 = pd.DataFrame(data2)

#Zipping the column of Date with Start time, Max time and End time
df2['Start_DateTime'] = list(zip(df2.iloc[:,0], df2.iloc[:,1]))
df2['CME_DateTime'] = list(zip(df2.iloc[:,9], df2.iloc[:,10]))
df2['End_DateTime'] = list(zip(df2.iloc[:,2], df2.iloc[:,3]))

#Drop the unnecessary columns
df2 = df2.drop('Start_Date', axis = 1)
df2 = df2.drop('End_Date', axis = 1)
df2 = df2.drop('Start_Time', axis = 1)
df2 = df2.drop('End_Time', axis = 1)
df2 = df2.drop('CME_Time', axis = 1)
df2 = df2.drop('CME_Date', axis = 1)

#Get rid of cells has unknow or undefined data
df2[df2=='????'] = 'NaN'
df2[df2=='BACK'] = 'NaN'
df2[df2=='-----'] = 'NaN'
df2[df2=='----'] = 'NaN'
df2[df2=='------'] = 'NaN'
df2[df2=='---'] = 'NaN'
df2[df2=='DSF'] = 'NaN'
df2[df2=='FILA'] = 'NaN'

df2.head(518)

,Start_Freq,End_Freq,Flare_Location,Flare_region,Importance,CPA,Width,Speed,Plots,is_Halo,WidthColumn,Start_DateTime,CME_DateTime,End_DateTime
0,8000,4000,S25E16,8026,M1.3,74,79,312,PHTX,False,False,"(1997-04-01, 14:00:00)","(1997-04-01, 15:18:00)","(1997-04-01, 14:15:00)"
1,11000,1000,S28E19,8027,C6.8,NaN,360,878,PHTX,True,False,"(1997-04-07, 14:30:00)","(1997-04-07, 14:27:00)","(1997-04-07, 17:30:00)"
2,12000,80,N21W08,8038,C1.3,NaN,360,464,PHTX,True,False,"(1997-05-12, 05:15:00)","(1997-05-12, 05:30:00)","(1997-05-14, 16:00:00)"
3,5000,500,N05W12,8040,M1.3,263,165,296,PHTX,False,False,"(1997-05-21, 20:20:00)","(1997-05-21, 21:00:00)","(1997-05-21, 22:00:00)"
4,6000,2000,S29E25,8088,C1.4,133,155,712,PHTX,False,False,"(1997-09-23, 21:53:00)","(1997-09-23, 22:02:00)","(1997-09-23, 22:16:00)"
5,14000,250,S20W13,8100,C8.6,240,109,227,PHTX,False,False,"(1997-11-03, 05:15:00)","(1997-11-03, 05:28:00)","(1997-11-03, 12:00:00)"
6,14000,5000,S16W21,8100,M4.2,233,122,352,PHTX,False,False,"(1997-11-03, 10:30:00)","(1997-11-03, 11:11:00)","(1997-11-03, 11:30:00)"
7,14000,100,S14W33,8100,X2.1,NaN,360,785,PHTX,True,False,"(1997-11-04, 06:00:00)","(1997-11-04, 06:10:00)","(1997-11-05, 04:30:00)"
8,14000,100,S18W63,8100,X9.4,NaN,360,1556,PHTX,True,False,"(1997-11-06, 12:20:00)","(1997-11-06, 12:10:00)","(1997-11-07, 08:30:00)"
9,14000,7000,N17E63,8113,X2.6,98,91,441,PHTX,False,False,"(1997-11-27, 13:30:00)","(1997-11-27, 13:56:00)","(1997-11-27, 14:00:00)"


Here we use map Clas_ to get the value of these alphabet from Wikipedia 
https://en.wikipedia.org/wiki/Solar_flare?fbclid=IwAR0VrK-Wc_dJtGAF42CjskjSg6_XbQdBF9_u1bWU3_YvhpAk5IKQwwmAfc0#Classification
then we sort the rows of the data frame based on temporary column
in this temporary column we calculate the value of each cell in importance column 
to sort the data frame based on this column
then we get the top 50

In [9]:
Clas_= {'X': 0.0001,
       'M': 0.00001, 
       'C': 0.000001, 
       'B': 0.0000001, 
       'A': 0.00000001}

MyList = []
for i in df2['Importance']:
    if i == 'NaN':
        MyList.append(0)
    else:
        MyList.append(Clas_[i[0]] * float(i[1:3]))

df2['New'] = MyList
df2 = df2.sort_values(by=['New'], ascending=False)
df2 = df2.drop('New', axis = 1)
New_df2 = df2[0:50]

New_df2.head(50)

,Start_Freq,End_Freq,Flare_Location,Flare_region,Importance,CPA,Width,Speed,Plots,is_Halo,WidthColumn,Start_DateTime,CME_DateTime,End_DateTime
240,10000,200,S19W83,10486,X28.,NaN,360,2657,PHTX,True,False,"(2003-11-04, 20:00:00)","(2003-11-04, 19:54:00)","(2003-11-04, 00:00:00)"
117,14000,250,N19W72,9393,X20.,261,244,2505,PHTX,False,False,"(2001-04-02, 22:05:00)","(2001-04-02, 22:06:00)","(2001-04-03, 02:30:00)"
233,14000,40,S16E08,10486,X17.,NaN,360,2459,PHTX,True,False,"(2003-10-28, 11:10:00)","(2003-10-28, 11:30:00)","(2003-10-29, 00:00:00)"
126,14000,40,S20W85,9415,X14.,245,167,1199,PHTX,False,False,"(2001-04-15, 14:05:00)","(2001-04-15, 14:06:00)","(2001-04-16, 13:00:00)"
234,11000,500,S15W02,10486,X10.,NaN,360,2029,PHTX,True,False,"(2003-10-29, 20:55:00)","(2003-10-29, 20:54:00)","(2003-10-29, 00:00:00)"
328,14000,250,S07E68,10930,X9.0,NaN,NaN,NaN,PHTX,False,False,"(2006-12-05, 10:50:00)","(NaN, NaN)","(2006-12-05, 20:00:00)"
8,14000,100,S18W63,8100,X9.4,NaN,360,1556,PHTX,True,False,"(1997-11-06, 12:20:00)","(1997-11-06, 12:10:00)","(1997-11-07, 08:30:00)"
514,16000,70,S08W33,12673,X9.3,NaN,360,1571,PHTX,True,False,"(2017-09-06, 12:05:00)","(2017-09-06, 12:24:00)","(2017-09-07, 08:00:00)"
515,16000,150,S09W92,NaN,X8.3,NaN,360,3163,PHTX,True,False,"(2017-09-10, 16:02:00)","(2017-09-10, 16:00:00)","(2017-09-11, 06:50:00)"
237,12000,250,S14W56,10486,X8.3,NaN,360,2598,PHTX,True,False,"(2003-11-02, 17:30:00)","(2003-11-02, 17:30:00)","(2003-11-03, 01:00:00)"


Matching is based on Date because its unique, the rest of the matches elements is not unique.
Matching can be done using Flare region also but Date is the best unique

In [25]:
'''
Here we get best matches between the 2 data frames
'''
FindMatch = pd.DataFrame()
Primary_Rank = []


cnt = 0
for _, row in New_df2.iterrows():
    for _, row2 in df.iterrows():
        if row.Start_DateTime[0] == row2.Start_Time[0]:
            row10 = pd.DataFrame([row])
            #We append the rank of the matches from first Data Frame to add it as column in the New DF
            Primary_Rank.append(row2.Rank)
            FindMatch = pd.concat([FindMatch, row10], ignore_index=True)
            cnt = cnt + 1
            break

#Add the column to the Best matches Data Frame
FindMatch['New Rank'] = Primary_Rank
FindMatch.head(cnt)

,Start_Freq,End_Freq,Flare_Location,Flare_region,Importance,CPA,Width,Speed,Plots,is_Halo,WidthColumn,Start_DateTime,CME_DateTime,End_DateTime,New Rank
0,10000,200,S19W83,10486,X28.,NaN,360,2657,PHTX,True,False,"(2003-11-04, 20:00:00)","(2003-11-04, 19:54:00)","(2003-11-04, 00:00:00)",1
1,14000,250,N19W72,9393,X20.,261,244,2505,PHTX,False,False,"(2001-04-02, 22:05:00)","(2001-04-02, 22:06:00)","(2001-04-03, 02:30:00)",2
2,14000,40,S16E08,10486,X17.,NaN,360,2459,PHTX,True,False,"(2003-10-28, 11:10:00)","(2003-10-28, 11:30:00)","(2003-10-29, 00:00:00)",3
3,14000,40,S20W85,9415,X14.,245,167,1199,PHTX,False,False,"(2001-04-15, 14:05:00)","(2001-04-15, 14:06:00)","(2001-04-16, 13:00:00)",5
4,11000,500,S15W02,10486,X10.,NaN,360,2029,PHTX,True,False,"(2003-10-29, 20:55:00)","(2003-10-29, 20:54:00)","(2003-10-29, 00:00:00)",6
5,14000,250,S07E68,10930,X9.0,NaN,NaN,NaN,PHTX,False,False,"(2006-12-05, 10:50:00)","(NaN, NaN)","(2006-12-05, 20:00:00)",9
6,14000,100,S18W63,8100,X9.4,NaN,360,1556,PHTX,True,False,"(1997-11-06, 12:20:00)","(1997-11-06, 12:10:00)","(1997-11-07, 08:30:00)",7
7,16000,70,S08W33,12673,X9.3,NaN,360,1571,PHTX,True,False,"(2017-09-06, 12:05:00)","(2017-09-06, 12:24:00)","(2017-09-07, 08:00:00)",8
8,16000,150,S09W92,NaN,X8.3,NaN,360,3163,PHTX,True,False,"(2017-09-10, 16:02:00)","(2017-09-10, 16:00:00)","(2017-09-11, 06:50:00)",11
9,12000,250,S14W56,10486,X8.3,NaN,360,2598,PHTX,True,False,"(2003-11-02, 17:30:00)","(2003-11-02, 17:30:00)","(2003-11-03, 01:00:00)",10


Plotting the relation between the start frequency in the top 50 data extracted from Nasa

In [1]:
Plot_Time = []
for i in New_df2.Start_DateTime:
    Plot_Time.append(i[1])
    
figure(figsize = (40, 10))
y_pos = np.arange(len(Plot_Time))
plt.bar(y_pos, New_df2.Start_Freq , align='center', alpha=1)
plt.xticks(y_pos, Plot_Time)
plt.ylabel('Start Frequency')
plt.title('Plotting Start Frequency ')

plt.show()

NameError: name 'New_df2' is not defined